In [3]:
pip install pandas transformers


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb_Hulk and Fat Thor.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(100, 2)


In [6]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': '1', 'score': 0.7729388475418091},
 {'label': '1', 'score': 0.7220338582992554},
 {'label': '2', 'score': 0.9447534680366516},
 {'label': '0', 'score': 0.49389761686325073},
 {'label': '1', 'score': 0.34969207644462585},
 {'label': '2', 'score': 0.7932775020599365},
 {'label': '1', 'score': 0.7875503897666931},
 {'label': '2', 'score': 0.6597854495048523},
 {'label': '2', 'score': 0.5819969773292542},
 {'label': '2', 'score': 0.7350302934646606},
 {'label': '2', 'score': 0.5544764995574951},
 {'label': '3', 'score': 0.3738649785518646},
 {'label': '2', 'score': 0.83124840259552},
 {'label': '2', 'score': 0.640611469745636},
 {'label': '2', 'score': 0.47339293360710144},
 {'label': '0', 'score': 0.824225902557373},
 {'label': '1', 'score': 0.5529389977455139},
 {'label': '2', 'score': 0.8306252956390381},
 {'label': '2', 'score': 0.7540708184242249},
 {'label': '1', 'score': 0.8129606246948242},
 {'label': '1', 'score': 0.7006308436393738},
 {'label': '3', 'score': 0.50499463

In [9]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,miek is so cute!!!,1,0.772939
1,"Kong playing PlayStation 4 slim, Noobmaster69 ...",1,0.722034
2,Who&#39;s here because you loved Fat Thor inst...,2,0.944753
3,I bet Thor had to threaten Kang for calling Ko...,0,0.493898
4,"<a href=""https://www.youtube.com/watch?v=Pzp_i...",1,0.349692
...,...,...,...
95,Bro you literally cut out the best part,2,0.528539
96,Just realized the god of thunder exclaiming &q...,2,0.632295
97,"<a href=""https://www.youtube.com/watch?v=Pzp_i...",1,0.401803
98,Fun Fact: Chris Hemsworth Is wearing a fat suit!,1,0.760830


In [10]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         1            joy
1         1            joy
2         2           love
3         0        sadness
4         1            joy
..      ...            ...
95        2           love
96        2           love
97        1            joy
98        1            joy
99        2           love

[100 rows x 2 columns]


<ipython-input-10-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [11]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_Hulk and Fat Thor_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,miek is so cute!!!,1,0.772939,joy
1,"Kong playing PlayStation 4 slim, Noobmaster69 ...",1,0.722034,joy
2,Who&#39;s here because you loved Fat Thor inst...,2,0.944753,love
3,I bet Thor had to threaten Kang for calling Ko...,0,0.493898,sadness
4,"<a href=""https://www.youtube.com/watch?v=Pzp_i...",1,0.349692,joy
...,...,...,...,...
95,Bro you literally cut out the best part,2,0.528539,love
96,Just realized the god of thunder exclaiming &q...,2,0.632295,love
97,"<a href=""https://www.youtube.com/watch?v=Pzp_i...",1,0.401803,joy
98,Fun Fact: Chris Hemsworth Is wearing a fat suit!,1,0.760830,joy
